In [1]:
from utils import optimizer_utils as ou
import warnings


In [2]:

# Disable SSL warnings (optional but not recommended in production)
warnings.filterwarnings("ignore")

In [3]:

system_prompt = "you are a senior python developer."

input_path = "./resources/original_script/"
output_path = "./resources/optimized_script/"

extension = ".py"

create_unitary_tests = True

# "llama3.2" or "gpt-4o-mini" / "gpt-4-turbo" / "gpt-4"	...
model = "gpt-4-turbo" 
output_file_name = "optimized_gpt_unit_tests.py"
execution_mode = "online" # online / offline

# call optimizer
ou.optimize_script(input_path, extension, output_path, model, system_prompt, execution_mode,output_file_name, create_unitary_tests)


Processing file: ./resources/original_script/original_notebook.py
▶️ Using gpt-4-turbo via OpenAI API...


📝 **Notes from LLM:**

NOTE:
- The original code was corrected for PEP 8 style compliance and redundancy. Functions were extracted from the single function to enhance modularity and reusability.
- All functions now include specific docstrings compliant with PEP 257. Detailed unit tests for each function have been added to ensure results are as expected, fulfilling the necessity of unittests.
- Each function now operates as a part of the data processing pipeline, which is more efficient and cleaner.
- Comments explaining the functionality were added before complex logic actions to clarify their intentions.
- ImportError and numpy redundancies were resolved, ensuring no modules are imported multiple times.
- Exception handling wasn't explicitly added because the operations are intrinsically safe with given correct input data; however, working on heterogeneous input might require such handling which should be managed by the caller.
- Comprehensive unittesting ensures functionality can be validated independently.

✅ Optimized script saved to: ./resources/optimized_script/optimized_gpt_unit_tests.py


In [4]:
# script path
script_path = output_path + output_file_name

# run the optimized script to ensure that the code runs OK
ou.run_python_script(script_path)

✅ Script executed successfully.

